In [1]:
__author__ = 'Monika Soraisam'
__email__ = 'monika.soraisam@noirlab.edu'

# This notebook is meant for comparing dataselect results with that from the GOATS frontend (https://noirlab.atlassian.net/browse/GOATS-385)

In [2]:
import os
import sys
from pathlib import Path
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import datetime
import types

In [3]:
# Now import the DRAGONS libraries 
import astrodata
import gemini_instruments
from recipe_system import cal_service
from gempy.adlibrary import dataselect
from gempy.utils import logutils

from recipe_system.reduction.coreReduce import Reduce
from gempy.scripts import showpars
from gempy.utils.showrecipes import showrecipes
from gempy.utils.showrecipes import showprims

In [4]:
## prep the reduction folder
def prep_reduction_folder(data_root, obsid):
    reduction_path = Path(f"{data_root}/{obsid}/reduction")

    if not reduction_path.exists():
        os.mkdir(reduction_path.as_posix())
        print (f"directory for dragons reduction created")
    
    ## change the cwd to the reduction folder
    os.chdir(reduction_path.as_posix())
    print(f"Current working directory is: {os.getcwd()}")

    ## write the configuration file 
    mydb = "dragons_for_goats.db" 
    mydb_path = reduction_path.as_posix() + '/' + mydb
    print (mydb_path)
    
    dragons_rc = reduction_path.as_posix() + '/dragonsrc'
    print (dragons_rc)
    
    with open(dragons_rc, "w") as f:
        f.write("[calibs]\ndatabases = {0} get store".format(mydb_path))

    return dragons_rc, mydb_path


In [5]:
def generate_filelists(location, obsid):
    """
    Parameters
    ----------
    location: str
        Root folder where the Gemini data for a given target is located
    obsid: str
        Gemini observation ID 
    """

    
    all_files = [str(pp) for pp in list(Path(location+"/"+obsid).glob('*.fits'))]
    all_files.sort()
    print (f'The total number of files for observation ID {obsid} is {len(all_files)}')

    obs_types = ['OBJECT','BIAS','DARK','FLAT','ARC','PINHOLE','RONCHI','CAL','FRINGE','MOS MASK', 'BPM'] #fetched from Obs Type search field on GOA, which is relevant for DRAGONS

    all_meta = {}

    # Note that even if a few keywords are extracted below, thiis is just meant for demonstration purposes. 
    # If onyl these keywords are extracted in production, it will not align with  
    # implementation of item (2), allowing users to group/select preferred header keywords
    for K in obs_types:
        all_meta[K] = {'file':[],
                        'obs_class':[],
                        'group_id':[],
                        'exp':[],
                        'object':[],
                        'wave':[],
                        'waveband':[],
                        'date':[],
                        'roi':[],
                        }

    object_files = []
    for i,F in enumerate(all_files):
        ad = astrodata.open(F)

        if "BPM" in ad.tags or "UNPREPARED" in ad.tags: ## want only raw, i.e., "unprepared" files, but BPM is an exception, which is processed/prepared 
            K = ad.observation_type() ## astrodata header also has the observation type, which should match what's in the archive drop-down menu
        
        elif "PREPARED" in ad.tags or "PROCESSED" in ad.tags: ## skip all "prepared"/"processed" files
            continue
            
        all_meta[K]['file'].append(F)
        all_meta[K]['obs_class'].append(ad.observation_class())
        
        # group_id seems to be not implemented for GNIRS spectroscopy yet
        if "GNIRS" in ad.instrument():
            all_meta[K]['group_id'].append(None)
        else:
            all_meta[K]['group_id'].append(ad.group_id())
        all_meta[K]['exp'].append(ad.exposure_time())
        all_meta[K]['object'].append(ad.object())
        all_meta[K]['wave'].append(ad.central_wavelength())
        all_meta[K]['waveband'].append(ad.wavelength_band())
        all_meta[K]['date'].append(ad.ut_date())
        all_meta[K]['roi'].append(ad.detector_roi_setting()) 
        #print (F.split('/')[-1], ad.object(), ad.tags)
    
    return all_meta


# GMOS spectroscopy

In [6]:
data_path = "/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM"
obsid = 'GS-2021A-DD-102-9'
dragons_rc, mydb_path = prep_reduction_folder(data_path, obsid)

directory for dragons reduction created
Current working directory is: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/reduction
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/reduction/dragons_for_goats.db
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/reduction/dragonsrc


In [7]:
all_meta = generate_filelists(data_path, obsid)

for K,V in all_meta.items():
    if len(V['file'])==0:
        continue
    print (f"There are {len(V['file'])} files for observation type {K}") 

The total number of files for observation ID GS-2021A-DD-102-9 is 113
There are 5 files for observation type OBJECT
There are 100 files for observation type BIAS
There are 3 files for observation type FLAT
There are 3 files for observation type ARC
There are 1 files for observation type BPM


In [8]:
DF_object = pd.DataFrame(all_meta['OBJECT'])
DF_bias = pd.DataFrame(all_meta['BIAS'])
DF_flat = pd.DataFrame(all_meta['FLAT'])
DF_arc = pd.DataFrame(all_meta['ARC'])
DF_bpm = pd.DataFrame(all_meta['BPM'])

In [9]:
print (f"There are {len(np.unique(DF_object['object'].values))} unique objects in this observation set, namely {np.unique(DF_object['object'].values)}")

There are 3 unique objects in this observation set, namely ['AT2020caa' 'LTT6248' 'Twilight']


## Results from grouping and using dataselect

In [13]:
for K, V in DF_object.groupby(['group_id','object']):
    print ("\n")
    print(K,len(V))
    for a in V['file'].values:
        print (a)



('GG455_6.5e-07_2_2_Normal_["\'BI5-36-4k-2, 4\':[1:512,1:4224]", "\'BI5-36-4k-2, 3\':[513:1024,1:4224]", "\'BI5-36-4k-2, 2\':[1025:1536,1:4224]", "\'BI5-36-4k-2, 1\':[1537:2048,1:4224]", "\'BI11-33-4k-1, 4\':[2049:2560,1:4224]", "\'BI11-33-4k-1, 3\':[2561:3072,1:4224]", "\'BI11-33-4k-1, 2\':[3073:3584,1:4224]", "\'BI11-33-4k-1, 1\':[3585:4096,1:4224]", "\'BI12-34-4k-1, 4\':[4097:4608,1:4224]", "\'BI12-34-4k-1, 3\':[4609:5120,1:4224]", "\'BI12-34-4k-1, 2\':[5121:5632,1:4224]", "\'BI12-34-4k-1, 1\':[5633:6144,1:4224]"]_R400', 'Twilight') 1
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210218S0029.fits


('GS-2021A-DD-102-5_GG455_6.5e-07_2_2_Normal_["\'BI5-36-4k-2, 4\':[1:512,1625:2648]", "\'BI5-36-4k-2, 3\':[513:1024,1625:2648]", "\'BI5-36-4k-2, 2\':[1025:1536,1625:2648]", "\'BI5-36-4k-2, 1\':[1537:2048,1625:2648]", "\'BI11-33-4k-1, 4\':[2049:2560,1625:2648]", "\'BI11-33-4k-1, 3\':[2561:3072,1625:2648]", "\'BI11-33-4k-1, 2\':[3073:358

In [17]:
filtered = dataselect.select_data(list(DF_object['file'].values), [], [], dataselect.expr_parser('central_wavelength<6.6E-7'))
filtered

['/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210218S0029.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0075.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0076.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0084.fits']

In [39]:
filtered = dataselect.select_data(list(DF_object['file'].values), [], [], dataselect.expr_parser('central_wavelength>6.5E-7'))
filtered

['/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0079.fits']

In [40]:
filtered = dataselect.select_data(list(DF_object['file'].values), [], [], dataselect.expr_parser('central_wavelength==6.5E-7'))
filtered

['/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210218S0029.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0075.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0076.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0084.fits']

In [41]:
filtered = dataselect.select_data(list(DF_object['file'].values), [], [], dataselect.expr_parser('central_wavelength==6.6E-7'))
filtered

['/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0079.fits']

In [44]:
for K, V in DF_object.groupby('exp'):
    print ("\n")
    print(K,len(V))
    for a in V['file'].values:
        print (a)



5.0 1
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210218S0029.fits


60.0 1
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0084.fits


900.0 3
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0075.fits
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0076.fits
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0079.fits


In [51]:
filtered = dataselect.select_data(list(DF_object['file'].values), [], [], dataselect.expr_parser('exposure_time==900.01', strict=True))
filtered

[]

In [52]:
filtered = dataselect.select_data(list(DF_object['file'].values), [], [], dataselect.expr_parser('exposure_time==900.01', strict=False))
filtered

['/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0075.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0076.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0079.fits']

In [53]:
filtered = dataselect.select_data(list(DF_object['file'].values), [], [], dataselect.expr_parser('exposure_time==900.0'))
filtered

['/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0075.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0076.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0079.fits']

In [46]:
# now let's also apply a filter to individual group
for K, V in DF_bias.groupby('group_id'):
    print ("\n")
    print(K,len(V))
    filtered = dataselect.select_data(list(V['file'].values), [], [], dataselect.expr_parser('ut_date>"2021-02-18" and ut_date<"2021-02-20"'))
    for a in filtered:
        print (a)



2_2_Normal_["'BI5-36-4k-2, 4':[1:512,1625:2648]", "'BI5-36-4k-2, 3':[513:1024,1625:2648]", "'BI5-36-4k-2, 2':[1025:1536,1625:2648]", "'BI5-36-4k-2, 1':[1537:2048,1625:2648]", "'BI11-33-4k-1, 4':[2049:2560,1625:2648]", "'BI11-33-4k-1, 3':[2561:3072,1625:2648]", "'BI11-33-4k-1, 2':[3073:3584,1625:2648]", "'BI11-33-4k-1, 1':[3585:4096,1625:2648]", "'BI12-34-4k-1, 4':[4097:4608,1625:2648]", "'BI12-34-4k-1, 3':[4609:5120,1625:2648]", "'BI12-34-4k-1, 2':[5121:5632,1625:2648]", "'BI12-34-4k-1, 1':[5633:6144,1625:2648]"] 50
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0165.fits
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0166.fits
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0167.fits
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0168.fits
/Users/monika.so

In [16]:
# now let's also apply a filter to individual group
for K, V in DF_bias.groupby('group_id'):
    print ("\n")
    print(K,len(V))
    filtered = dataselect.select_data(list(V['file'].values), [], [], dataselect.expr_parser('ut_date>"2021-02-18" and ut_date<"2021-02-20" and detector_roi_setting=="Full Frame"'))
    for a in filtered:
        print (a)



2_2_Normal_["'BI5-36-4k-2, 4':[1:512,1625:2648]", "'BI5-36-4k-2, 3':[513:1024,1625:2648]", "'BI5-36-4k-2, 2':[1025:1536,1625:2648]", "'BI5-36-4k-2, 1':[1537:2048,1625:2648]", "'BI11-33-4k-1, 4':[2049:2560,1625:2648]", "'BI11-33-4k-1, 3':[2561:3072,1625:2648]", "'BI11-33-4k-1, 2':[3073:3584,1625:2648]", "'BI11-33-4k-1, 1':[3585:4096,1625:2648]", "'BI12-34-4k-1, 4':[4097:4608,1625:2648]", "'BI12-34-4k-1, 3':[4609:5120,1625:2648]", "'BI12-34-4k-1, 2':[5121:5632,1625:2648]", "'BI12-34-4k-1, 1':[5633:6144,1625:2648]"] 50


2_2_Normal_["'BI5-36-4k-2, 4':[1:512,1:4224]", "'BI5-36-4k-2, 3':[513:1024,1:4224]", "'BI5-36-4k-2, 2':[1025:1536,1:4224]", "'BI5-36-4k-2, 1':[1537:2048,1:4224]", "'BI11-33-4k-1, 4':[2049:2560,1:4224]", "'BI11-33-4k-1, 3':[2561:3072,1:4224]", "'BI11-33-4k-1, 2':[3073:3584,1:4224]", "'BI11-33-4k-1, 1':[3585:4096,1:4224]", "'BI12-34-4k-1, 4':[4097:4608,1:4224]", "'BI12-34-4k-1, 3':[4609:5120,1:4224]", "'BI12-34-4k-1, 2':[5121:5632,1:4224]", "'BI12-34-4k-1, 1':[5633:6144,1

In [21]:
# now let's also apply a filter to individual group
for K, V in DF_bias.groupby('group_id'):
    print ("\n")
    print(K,len(V))
    filtered = dataselect.select_data(list(V['file'].values), [], [], dataselect.expr_parser('ut_date>"2021-02-18" and ut_date<"2021-02-20" and detector_roi_setting=="Central Spectrum"'))
    for a in filtered:
        print (a)



2_2_Normal_["'BI5-36-4k-2, 4':[1:512,1625:2648]", "'BI5-36-4k-2, 3':[513:1024,1625:2648]", "'BI5-36-4k-2, 2':[1025:1536,1625:2648]", "'BI5-36-4k-2, 1':[1537:2048,1625:2648]", "'BI11-33-4k-1, 4':[2049:2560,1625:2648]", "'BI11-33-4k-1, 3':[2561:3072,1625:2648]", "'BI11-33-4k-1, 2':[3073:3584,1625:2648]", "'BI11-33-4k-1, 1':[3585:4096,1625:2648]", "'BI12-34-4k-1, 4':[4097:4608,1625:2648]", "'BI12-34-4k-1, 3':[4609:5120,1625:2648]", "'BI12-34-4k-1, 2':[5121:5632,1625:2648]", "'BI12-34-4k-1, 1':[5633:6144,1625:2648]"] 50
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0165.fits
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0166.fits
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0167.fits
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2021A-DD-102-9/S20210219S0168.fits
/Users/monika.so

# F2 imaging

In [22]:
data_path = "/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM"
obsid = 'GS-2015B-Q-46-59'  
dragons_rc, mydb_path = prep_reduction_folder(data_path, obsid)

directory for dragons reduction created
Current working directory is: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/reduction
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/reduction/dragons_for_goats.db
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/reduction/dragonsrc


In [23]:
all_meta = generate_filelists(data_path, obsid)

for K,V in all_meta.items():
    if len(V['file'])==0:
        continue
    print (f"There are {len(V['file'])} files for observation type {K}") 

The total number of files for observation ID GS-2015B-Q-46-59 is 298
There are 156 files for observation type OBJECT
There are 110 files for observation type DARK
There are 32 files for observation type FLAT


In [24]:
DF_object = pd.DataFrame(all_meta['OBJECT'])
DF_dark = pd.DataFrame(all_meta['DARK'])
DF_flat = pd.DataFrame(all_meta['FLAT'])

In [25]:
print (f"There are {len(np.unique(DF_object['object'].values))} unique objects in this observation set, namely {np.unique(DF_object['object'].values)}")

There are 2 unique objects in this observation set, namely ['NGC 2244' 'Rubin 149 (ukirt fs14)']


## Results from grouping and using dataselect

In [27]:
for K, V in DF_object.groupby(['group_id','object']):
    print ("\n")
    print(K,len(V))
    filtered = dataselect.select_data(list(V['file'].values), [], [], dataselect.expr_parser('filter_name=="Ks"'))
    for a in filtered:
        print (a)    



("GS-2015B-Q-46-59_H_f/16_G5830_13.0_1_['[1:2048,1:2048]']", 'NGC 2244') 128


("GS-2015B-Q-46-59_Ks_f/16_G5830_15.0_1_['[1:2048,1:2048]']", 'NGC 2244') 22
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151128S0369.fits
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151128S0370.fits
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151128S0371.fits
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151128S0372.fits
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151128S0373.fits
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151128S0374.fits
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151128S0375.fits
/Users/monika.soraisam/Desktop/tomdev/real_g

In [28]:
filtered = dataselect.select_data(list(DF_object['file'].values), [], [], dataselect.expr_parser('filter_name=="H" and object=="Rubin 149 (ukirt fs14)"'))
filtered

['/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151129S0092.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151129S0093.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151129S0094.fits']

In [72]:
filtered = dataselect.select_data(list(DF_object['file'].values), [], [], dataselect.expr_parser('filter_name=="Ks" and object=="Rubin 149 (ukirt fs14)"'))
filtered

['/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151129S0095.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151129S0096.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151129S0097.fits']

In [67]:
filtered = dataselect.select_data(list(DF_dark['file'].values), [], [], dataselect.expr_parser('exposure_time==2.0'))
filtered

['/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151128S0511.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151128S0512.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151128S0513.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151128S0514.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151128S0515.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151128S0516.fits',
 '/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18acppvil/GEM/GS-2015B-Q-46-59/S20151128S0517.fits']